## चरण 3: Syft Keras का उपयोग करते हुए निजी पूर्वानुमान - सेवा (ग्राहक)

अनुवादक - nbTranslate 

संपादक - Urvashi Raheja - Github: [@raheja](https://github.com/raheja)


बधाई हो! सामान्य केरस के साथ अपने मॉडल को प्रशिक्षित करने और इसे सैफ्ट केरस के साथ सुरक्षित करने के बाद, आप कुछ निजी भविष्यवाणियों का अनुरोध करने के लिए तैयार हैं।

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist

import syft as sy

## डेटा

यहां, हम अपने MNIST डेटा को प्रीप्रोसेस करते हैं। यह उसी तरह है जैसे हमने प्रशिक्षण के दौरान कैसे किया।

In [2]:
# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

## मॉडल से कनेक्ट करें

मॉडल को क्वेरी करने से पहले, आपको बस इसे कनेक्ट करना होगा। ऐसा करने के लिए, आप एक क्लाइंट बना सकते हैं। फिर ठीक उसी तीन TFEWorkers (`alice` , `bob` , और `carol`) और क्लस्टर को परिभाषित करें। अंत में `connect_to_model` को कॉल करें। यह क्लाइंट मॉडल साइड पर एक TFE कतारबद्ध सर्वर बनाता है जो कि `model.serve ()` द्वारा स्थापित कतार सर्वर से जोड़ता है **भाग 13b**। शेयरों को एक पूर्वानुमान अनुरोध में शेयरों को जमा करने से पहले सादा डेटा साझा करने की गुप्त रूप से ज़िम्मेदारी होगी।

In [3]:
num_classes = 10
input_shape = (1, 28, 28, 1)
output_shape = (1, num_classes)

In [4]:
client = sy.TFEWorker()

alice = sy.TFEWorker(host='localhost:4000')
bob = sy.TFEWorker(host='localhost:4001')
carol = sy.TFEWorker(host='localhost:4002')
cluster = sy.TFECluster(alice, bob, carol)

client.connect_to_model(input_shape, output_shape, cluster)

INFO:tf_encrypted:Starting session on target 'grpc://localhost:4000' using config graph_options {
}



## क्वेरी मॉडल

आप कुछ निजी भविष्यवाणियां करने के लिए तैयार हैं! `query_model` को कॉल करने से ऊपर बनाई गई कतार में `image` सम्मिलित हो जाएगी, गुप्त रूप से डेटा को स्थानीय रूप से साझा किया जाएगा, और मॉडल सर्वर में शेयरों को **भाग 13 बी** में सबमिट किया जाएगा।

In [5]:
# User inputs
num_tests = 3
images, expected_labels = x_test[:num_tests], y_test[:num_tests]

In [6]:
for image, expected_label in zip(images, expected_labels):

    res = client.query_model(image.reshape(1, 28, 28, 1))
    predicted_label = np.argmax(res)

    print("The image had label {} and was {} classified as {}".format(
        expected_label,
        "correctly" if expected_label == predicted_label else "wrongly",
        predicted_label))

The image had label 7 and was correctly classified as 7
The image had label 2 and was correctly classified as 2
The image had label 1 and was correctly classified as 1


यह भी खूब रही। आप इन तीन छवियों को सही ढंग से वर्गीकृत करने में सक्षम हैं! लेकिन इन पूर्वानुमान के बारे में विशेष बात यह है कि आपने इस सेवा को प्राप्त करने के लिए किसी भी निजी जानकारी का खुलासा नहीं किया है। मॉडल होस्ट ने आपके इनपुट डेटा या आपकी भविष्यवाणियों को कभी नहीं देखा, और आपने कभी मॉडल डाउनलोड नहीं किया। आप एन्क्रिप्टेड मॉडल के साथ एन्क्रिप्टेड डेटा पर निजी पूर्वानुमान प्राप्त करने में सक्षम थे!

इससे पहले कि हम अपने स्वयं के ऐप्स में इसे लागू करने के लिए जल्दी से आगे बढ़ें, चलो जल्दी से अपने सेवा मॉडल को साफ करने के लिए **भाग 13 बी** पर जाएं!